In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf

In [6]:
os.listdir("./img")

['New folder']

In [4]:
images = os.listdir("./retinopathy detection dataset/")[10]

In [5]:
images

'train1'

In [6]:
train_labels = pd.read_csv('./retinopathy detection dataset/trainLabels.csv')

In [7]:
train_labels.head()

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [8]:
np.size(os.listdir("./retinopathy detection dataset/train_1"))

5531

In [9]:
from PIL import Image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image

Using TensorFlow backend.


In [10]:
lst = os.listdir("./retinopathy detection dataset/train_1")
np.size(lst)

5531

In [ ]:
immatrix = []
imlabels = []

In [ ]:
rows,cols = 256,256

In [ ]:
import cv2
from tqdm import tqdm

for file in tqdm(lst):
    base = os.path.basename("./retinopathy detection dataset/train_1/" + file)
    print(file)
    fileName = os.path.splitext(base)[0]
    imlabels.append(train_labels.loc[train_labels.image==fileName, 'level'].values[0])
    im = cv2.imread("./retinopathy detection dataset/train_1/" + file)   
    img = cv2.resize(im,(rows,cols))
    immatrix.append(np.array(img))

In [ ]:
immatrix = np.asarray(immatrix)
imlabels = np.asarray(imlabels)

In [ ]:
imlabels[:50]

In [ ]:
immatrix.shape

In [ ]:
imlabels.shape

In [ ]:
from sklearn.utils import shuffle
with tf.device("/gpu:0"):
    data,Label = shuffle(immatrix,imlabels, random_state=2)
    train_data = [data,Label]
    type(train_data)

In [ ]:
(X,y) = (train_data[0],train_data[1])

In [ ]:
from sklearn.model_selection import train_test_split
with tf.device("/gpu:0"):
    X_train, X_test, y_train, y_test = train_test_split(X[1:35124], y[1:35124], test_size=0.2, random_state=4)

    print(X_train.shape)
    print(X_test.shape)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    #Normalizing Data Between [0,1]
    X_train /= 255
    X_test /= 255

    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    
    

In [ ]:
y_train[0]

In [ ]:
X_train[0].shape

In [ ]:
from keras.utils import np_utils

n_classes = 5

Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

In [ ]:
Y_train

In [ ]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))
print(base_model.summary())

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Flatten , Dropout, Activation

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(5, activation='softmax'))

In [ ]:
from keras.models import Model

model=Model(inputs=base_model.input, outputs=top_model(base_model.output))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# create generators  - training data will be augmented images
validationdatagenerator = ImageDataGenerator()
traindatagenerator = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,rotation_range=15,zoom_range=0.1 )

batchsize=8
train_generator=traindatagenerator.flow(X_train, Y_train, batch_size=batchsize) 
validation_generator=validationdatagenerator.flow(X_test, Y_test,batch_size=batchsize)

In [ ]:
for layer in model.layers[:15]:
    layer.trainable = False

In [ ]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
     optimizer=optimizers.RMSprop(lr=1e-4),
     metrics=['accuracy'])
model.summary()

In [ ]:
batch_size =32
nb_epochs =10
nb_classes = 5

In [ ]:
with tf.device("/gpu:0"):
    model.fit(x = X_train,y=Y_train)

In [ ]:
with tf.device("/gpu:0"):
    model.fit_generator(train_generator, steps_per_epoch=int(len(X_train)/batchsize), 
                    epochs=6, validation_data=validation_generator, 
                    validation_steps=int(len(X_test)/batchsize))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_train_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("DR_train_1.h5")
print("Saved model to disk")

In [ ]:
with tf.device("/gpu:0"):
    probs = model.predict(X_test)

In [ ]:
probs[:10]

In [ ]:
Y_test[:10]